# LangChain Expression Language (LCEL)

El Lenguaje de Expresión de Cadenas de LangChain, o LCEL, es una manera declarativa de componer fácilmente cadenas juntas. El LCEL fue diseñado desde el primer día para admitir la implementación de prototipos en producción, sin cambios de código, desde la cadena más simple de "prompt + LLM" hasta las cadenas más complejas (hemos visto a personas ejecutar con éxito cadenas LCEL con 100s de pasos en producción). Para resaltar algunas de las razones por las que podrías querer usar LCEL:

Soporte de transmisión: al construir tus cadenas con LCEL, obtienes el mejor tiempo posible hasta el primer token (tiempo transcurrido hasta que sale el primer fragmento de salida). Para algunas cadenas, esto significa, por ejemplo, transmitir tokens directamente desde un LLM a un analizador de salida en continuo, y obtener fragmentos analizados e incrementales de salida a la misma velocidad que el proveedor de LLM emite los tokens en bruto.

Soporte asíncrono: cualquier cadena construida con LCEL puede ser llamada tanto con la API síncrona (por ejemplo, en tu cuaderno Jupyter durante la prototipificación) como con la API asíncrona (por ejemplo, en un servidor LangServe). Esto permite utilizar el mismo código para prototipos y en producción, con un rendimiento excelente y la capacidad de manejar muchas solicitudes concurrentes en el mismo servidor.

Ejecución paralela optimizada: siempre que las cadenas LCEL tengan pasos que se puedan ejecutar en paralelo (por ejemplo, si recuperas documentos de varios recuperadores), lo hacemos automáticamente, tanto en las interfaces síncronas como asíncronas, para la latencia más pequeña posible.

Reintentos y alternativas: configura reintentos y alternativas para cualquier parte de tu cadena LCEL. Esta es una excelente manera de hacer que tus cadenas sean más confiables a escala. Actualmente estamos trabajando en agregar soporte de transmisión para reintentos/alternativas, para que puedas obtener confiabilidad adicional sin costo de latencia.

Acceso a resultados intermedios: para cadenas más complejas, a menudo es muy útil acceder a los resultados de pasos intermedios incluso antes de que se produzca la salida final. Esto se puede usar para informar a los usuarios finales que algo está sucediendo, o incluso solo para depurar tu cadena. Puedes transmitir resultados intermedios, y está disponible en cada servidor LangServe.

Esquemas de entrada y salida: los esquemas de entrada y salida proporcionan a cada cadena LCEL esquemas Pydantic y JSONSchema inferidos a partir de la estructura de tu cadena. Esto se puede usar para la validación de entradas y salidas, y es una parte integral de LangServe.

Integración transparente con el seguimiento de LangSmith: a medida que tus cadenas se vuelven más y más complejas, es cada vez más importante entender qué está sucediendo exactamente en cada paso. Con LCEL, todos los pasos se registran automáticamente en LangSmith para una máxima observabilidad y capacidad de depuración.

Integración transparente con el despliegue de LangServe: cualquier cadena creada con LCEL se puede implementar fácilmente utilizando LangServe.

# Comenzar
LCEL facilita la construcción de cadenas complejas a partir de componentes básicos y admite funcionalidades listas para usar, como transmisión, paralelismo y registro.

## Ejemplo básico: prompt + modelo + analizador de salida
El caso de uso más básico y común es encadenar una plantilla de prompt y un modelo. Para ver cómo funciona esto, creemos una cadena que tome un tema y genere un chiste:


In [1]:
#%pip install --upgrade --quiet langchain-core langchain-community langchain-openai

# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Cuentame un chiste de {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo-0125")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "procrastinación"})

'¿Sabes cuál es el mejor momento para hacer una tarea que has estado posponiendo durante semanas? ¡Mañana!'

Observa esta línea de código, donde ensamblamos diferentes componentes en una única cadena utilizando LCEL:

In [3]:
chain = prompt | model | output_parser

El símbolo | es similar a un operador de tubería de Unix, que encadena los diferentes componentes alimentando la salida de un componente como entrada al siguiente componente.

En esta cadena, la entrada del usuario se pasa a la plantilla de prompt, luego la salida de la plantilla de prompt se pasa al modelo, y finalmente, la salida del modelo se pasa al analizador de salida. Echemos un vistazo a cada componente individualmente para entender realmente lo que está sucediendo.

## 1.  Prompt
`prompt` es un `BasePromptTemplate`, lo que significa que toma un diccionario de variables de plantilla y produce un `PromptValue`. Un `PromptValue` es un contenedor alrededor de un prompt completado que se puede pasar tanto a un LLM (que toma una cadena como entrada) como a un ChatModel (que toma una secuencia de mensajes como entrada). Puede trabajar con ambos tipos de modelos de lenguaje porque define lógica tanto para producir `BaseMessages` como para producir una cadena.

In [4]:
prompt_value = prompt.invoke({"topic": "Nieve Napolitana"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='Cuentame un chiste de Nieve Napolitana')])

In [5]:
prompt_value.to_messages()

[HumanMessage(content='Cuentame un chiste de Nieve Napolitana')]

In [6]:
prompt_value.to_string()

'Human: Cuentame un chiste de Nieve Napolitana'

## 2.  Modelo 
El `PromptValue` se pasa al modelo. En este caso, nuestro modelo es un `ChatModel`, lo que significa que producirá un `BaseMessage` como salida.

In [7]:
message = model.invoke(prompt_value)
message

AIMessage(content='¿Cuál es el postre favorito de los esquimales? ¡La nieve napolitana!')

Si nuestro modelo fuera un LLM, generaría una cadena.

In [8]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\n¿Por qué la nieve napolitana siempre está tan fría?\n\nPorque siempre está en el congelador de la pizza. '

## 3.  Analizador de salida
Y finalmente, pasamos la salida de nuestro modelo al `output_parser`, que es un `BaseOutputParser`, lo que significa que toma como entrada tanto una cadena como un `BaseMessage`. El `StrOutputParser`, en particular, convierte de manera simple cualquier entrada en una cadena.

In [9]:
output_parser.invoke(message)

'¿Cuál es el postre favorito de los esquimales? ¡La nieve napolitana!'

## 4.  Toda la tubería (Pipeline completa)

Para seguir los pasos:

1.  Ingresamos la entrada del usuario sobre el tema deseado, por ejemplo, {"topic": "helado"}.
2.  El componente de prompt toma la entrada del usuario, que luego se utiliza para construir un `PromptValue` después de usar el tema para construir el prompt.
3.  El componente del modelo toma el prompt generado y lo pasa al modelo de lenguaje (LLM) de OpenAI para su evaluación. La salida generada por el modelo es un objeto `ChatMessage`.
4.  Finalmente, el componente de `output_parser` recibe un `ChatMessage` y lo transforma en una cadena de Python, que se devuelve desde el método de invocación.

Estructura:
-----------

*   `Dict`: Entrada: `topic=helado`
*   `PromptTemplate`: Componente de prompt
*   `ChatModel`: Componente de modelo
*   `StrOutputParser`: Componente de análisis de salida
*   Resultado: Cadena de salida

Ten en cuenta que si tienes curiosidad acerca de la salida de cualquier componente, siempre puedes probar una versión más pequeña de la cadena, como `prompt` o `prompt | model`, para ver los resultados intermedios.

In [10]:
input = {"topic": "helado"}

prompt.invoke(input)
# > ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

(prompt | model).invoke(input)
# > AIMessage(content="Why did the ice cream go to therapy?\nBecause it had too many toppings and couldn't cone-trol itself!")

AIMessage(content='¿Cuál es el helado más peligroso? El helado asesino, ¡porque mata de frío!')

# Ejemplo de Búsqueda RAG

Para nuestro próximo ejemplo, queremos ejecutar una cadena de generación aumentada con recuperación (RAG) para agregar contexto al responder preguntas.

In [11]:
%pip install langchain docarray tiktoken

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("¿Dónde trabajaba Harrison?")

'Harrison trabajaba en Kensho.'

En este caso, la cadena compuesta es:


In [13]:
chain = setup_and_retrieval | prompt | model | output_parser

Para explicar esto, primero podemos observar que la plantilla de prompt anterior toma el contexto y la pregunta como valores a ser sustituidos en el prompt. Antes de construir la plantilla de prompt, queremos recuperar documentos relevantes para la búsqueda e incluirlos como parte del contexto.

Como paso preliminar, hemos configurado el recuperador utilizando un almacenamiento en memoria, que puede recuperar documentos basados en una consulta. Este también es un componente ejecutable que se puede encadenar con otros componentes, pero también puedes intentar ejecutarlo por separado:

In [15]:
retriever.invoke("¿Dónde trabajaba Harrison?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

Luego, utilizamos el componente `RunnableParallel` para preparar las entradas esperadas en el prompt, utilizando las entradas para los documentos recuperados, así como la pregunta original del usuario. Utilizamos el recuperador para la búsqueda de documentos y `RunnablePassthrough` para pasar la pregunta del usuario:

In [16]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

Para repasar, la cadena completa es:


In [17]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

Con el flujo siendo:

1. Los primeros pasos crean un objeto RunnableParallel con dos entradas. La primera entrada, context, incluirá los resultados de los documentos recuperados por el recuperador. La segunda entrada, question, contendrá la pregunta original del usuario. Para pasar la pregunta, utilizamos RunnablePassthrough para copiar esta entrada.

2. Alimentamos el diccionario del paso anterior al componente de prompt. Luego, toma la entrada del usuario, que es la pregunta, así como el documento recuperado, que es el contexto, para construir un prompt y producir un PromptValue.

3. El componente del modelo toma el prompt generado y lo pasa al modelo de lenguaje (LLM) de OpenAI para su evaluación. La salida generada por el modelo es un objeto ChatMessage.

4. Finalmente, el componente de output_parser recibe un ChatMessage y lo transforma en una cadena de Python, que se devuelve desde el método de invocación.

# Pasos siguientes
Recomendamos leer nuestra sección ["Why use LCEL"](https://python.langchain.com/docs/expression_language/why) a continuación para ver una comparación lado a lado del código necesario para producir funcionalidades comunes con y sin LCEL.